In [177]:
import requests
import datetime as dt


def initial_request(symbol):
    """Query the Yahoo! Finance page for a particular ticker symbol.
    
    Args:
        symbol (str): The ticker symbol.
        
    Returns:
        requests.models.Response: The HTML of the requested page.
    """
    initial_url = "https://finance.yahoo.com/quote/" + symbol + "?p=" + symbol
    return requests.get(initial_url, verify=False)


def get_cookies(r):
    """Grab the cookies and crumb from a requested Yahoo! Finance page for a particular symbol.
    
    Args:
        r (requests.models.Response): The HTML of the requested page.
    
    Returns:
        (requests.cookies.RequestsCookieJar, str): The cookie jar and associated crumb.
    """
    cookies = r.cookies
    crumbStore_begin = r.text.find("CrumbStore")
    crumbStore_end = r.text.find("}", crumbStore_begin) - 1
    crumbStore = r.text[crumbStore_begin:crumbStore_end]
    crumb_begin = crumbStore.rfind("\"") + 1
    crumb = crumbStore[crumb_begin:]
    return (cookies, crumb)
    

def date_sse(date):
    """Translate a date into a number of seconds since epoch.
    
    Args:
        date (datetime.date): The date you'd like to translate.
    
    Returns:
        str: The number of seconds since the Unix epoch (1970-01-01 00:00:00).
    """
    # DO I NEED TO WORRY ABOUT TIMEZONES?
    import calendar
    date = dt.datetime.combine(date, dt.datetime.min.time())
    return str(calendar.timegm(date.utctimetuple()))


def final_request(symbol, start_date, end_date, cookies, crumb):
    """Get a CSV of the historical financial data for the specified ticker symbol.
    
    Args:
        symbol (str): The ticker symbol for which you'd like the historical data.
        start_date (datetime.date): The first day for which you'd like data.
        end_date (datetime.date): The last day for which you'd like data.
        cookies (requests.cookies.RequestsCookieJar): The cookie jar needed to make the request.
        crumb (str): The cookie crumb needed to make the request.
    
    Returns:
        str: A comma-separated string with column headings Date, Open, High, Low, Close, Adj Close, and Volume.
    """
    url = ("https://query1.finance.yahoo.com/v7/finance/download/"
           + symbol
           + "?period1="
           + date_sse(start_date)
           + "&period2="
           + date_sse(end_date)
           + "&interval=1d"
           + "&events=history"
           + "&crumb="
           + crumb)
    return requests.get(url, cookies=cookies, verify=False)


def get_data(symbol, start_date=dt.date.today(), end_date=dt.date.today()):
    """Get historical financial data for the given ticker symbol.
    
    Args:
        symbol (str): The ticker symbol for which you'd like the historical data.
        start_date (datetime.date, optional): The first day for which you'd like data. Defaults to today.
        end_date (datetime.date, optional): The last day for which you'd like data. Defaults to today.
    
    Returns:
        pandas.core.frame.DataFrame: The requested historical financial data.
    """
    import pandas as pd
    from io import StringIO
    symbol = symbol.upper()
    # CHECK TO ENSURE DATES ARE VALID?
    r = initial_request(symbol)
    (cookies, crumb) = get_cookies(r)
    data = final_request(symbol, start_date, end_date, cookies, crumb)
    return pd.read_csv(StringIO(data.text), index_col=0)

In [176]:
from pandas.tseries.offsets import *
get_data("GOOG", dt.date.today() - 7*BDay(), dt.date.today())

/Users/jmgate/Library/Enthought/Canopy/edm/envs/User/lib/python3.5/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/jmgate/Library/Enthought/Canopy/edm/envs/User/lib/python3.5/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-23,1159.849976,1171.626953,1158.750000,1169.969971,1169.969971,1333100
2018-01-24,1177.329956,1179.859985,1161.050049,1164.239990,1164.239990,1416600
2018-01-25,1172.530029,1175.939941,1162.760010,1170.369995,1170.369995,1480500
2018-01-26,1175.079956,1175.839966,1158.109985,1175.839966,1175.839966,2018800
2018-01-29,1176.479980,1186.890015,1171.979980,1175.579956,1175.579956,1378900
2018-01-30,1167.829956,1176.520020,1163.520020,1163.689941,1163.689941,1556300
2018-01-31,1170.569946,1173.000000,1159.130005,1169.939941,1169.939941,1512600


In [299]:
class YahooDataReader:
    """
    """
    import pandas as _pd
    from io import StringIO as _StringIO
    import requests as _requests
    import datetime as _dt
    import calendar as _cal

        
    def __init__(self, symbol):
        self.symbol = symbol
        self._validate_symbol()
        self._initial_request()
        self._get_cookies()
        self._initial_request = None
        self._cookies = None
        self._crumb = None
        self._start_date = None
        self._end_date = None
        self._request_results = None
        self.data = None
    
    
    def __repr__(self):
        return "<YahooDataReader: symbol:{} start_date:{} end_date:{}>".format(
            self.symbol,
            self._start_date,
            self._end_date)
    
    def __str__(self):
        return """YahooDataReader:
    Ticker Symbol:  {}
    Start Date:     {}
    End Date:       {}
        """.format(self.symbol, self._start_date, self._end_date)

        
    def _validate_symbol(self):
        """Ensure we have a valid ticker symbol."""
        self.symbol = self.symbol.upper()
        # CHECK TO MAKE SURE INPUT IS VALID TICKER SYMBOL.


    def get_data(self, start_date=_dt.date.today(), end_date=None):
        """Get historical financial data between two dates.

        Args:
            start_date (datetime.date, optional): The first day for which you'd like data. Defaults to today.
            end_date (datetime.date, optional): The last day for which you'd like data. Defaults to today.

        Returns:
            pandas.core.frame.DataFrame: The requested historical financial data.
        """
        self._validate_dates(start_date, end_date)
        self._final_request()
        self._get_dataframe()
    
    
    def _validate_dates(self, start_date, end_date):
        """
        """
        if not end_date:
            end_date = start_date
        # MAKE SURE END DATE IS AFTER OR EQUAL TO START DATE.
        self._start_date = start_date
        self._end_date = end_date
    
    
    def _initial_request(self):
        """Query the Yahoo! Finance page."""
        initial_url = ("https://finance.yahoo.com/quote/"
                       + self.symbol
                       + "?p="
                       + self.symbol)
        self._initial_request = requests.get(initial_url, verify=False)


    def _get_cookies(self):
        """Grab the cookies and crumb from a requested Yahoo! Finance page."""
        r = self._initial_request
        self._cookies = r.cookies
        crumb_store_begin = r.text.find("CrumbStore")
        crumb_store_end = r.text.find("}", crumb_store_begin) - 1
        crumb_store = r.text[crumb_store_begin:crumb_store_end]
        crumb_begin = crumb_store.rfind("\"") + 1
        self._crumb = crumb_store[crumb_begin:]


    def date_to_sse(date):
        """Translate a date into a number of seconds since epoch.

        Args:
            date (datetime.date): The date you'd like to translate.

        Returns:
            str: The number of seconds since the Unix epoch (1970-01-01 00:00:00).
        """
        # DO I NEED TO WORRY ABOUT TIMEZONES?
        date = _dt.datetime.combine(date, _dt.datetime.min.time())
        return str(_cal.timegm(date.utctimetuple()))


    def _final_request(self):
        """Get a CSV of the historical financial data.

        Args:
            start_date (datetime.date): The first day for which you'd like data.
            end_date (datetime.date): The last day for which you'd like data.
        """
        url = ("https://query1.finance.yahoo.com/v7/finance/download/"
               + self.symbol
               + "?period1="
               + date_to_sse(self._start_date)
               + "&period2="
               + date_to_sse(self._end_date)
               + "&interval=1d"
               + "&events=history"
               + "&crumb="
               + self._crumb)
        self._request_results = _requests.get(url, cookies=self._cookies, verify=False)
    
    
    def _get_dataframe(self):
        """Read the CSV results into a DataFrame."""
        self.data = _pd.read_csv(_StringIO(self._data.text), index_col=0)



In [300]:
test = YahooDataReader("goog")

/Users/jmgate/Library/Enthought/Canopy/edm/envs/User/lib/python3.5/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [301]:
test

<YahooDataReader: symbol:GOOG start_date:None end_date:None>

In [302]:
print(test)

YahooDataReader:
    Ticker Symbol:  GOOG
    Start Date:     None
    End Date:       None
        


In [303]:
first = dt.date.today()
last = first

In [304]:
test.get_data(first, last)

NameError: name 'date_to_sse' is not defined

In [274]:
%debug

> <ipython-input-268-18934ae7fa40>(56)get_data()
     54         """
     55         self._validate_dates(start_date, end_date)
---> 56         self._final_request()
     57         self._get_dataframe()
     58 

ipdb> self
<YahooDataReader: symbol:GOOG start_date:2018-02-01 end_date:2018-02-01>
ipdb> d
*** Newest frame
ipdb> q
